### Coin data pre-processing

Pre-process and combine coin tvl, volume and price data

In [3]:
import pandas as pd
import plotly.graph_objects as go
from glob import glob
import numpy as np
import plotly.express as px

In [17]:
coins = ["BTC" , "ETH", "SOL", "SUI"]

for coin in coins:
    # Read the CSV data into a DataFrame
    df = pd.read_csv("data/" + coin + "_TVL.csv")
    
    # Transpose the DataFrame
    df_transposed = df.transpose()
    
    # The first row will be the new header, so reset the header
    df_transposed.columns = df_transposed.iloc[0]
    
    # Drop the old header row
    df_transposed = df_transposed[1:]
    
    # Rename the index to 'Date'
    df_transposed.index.name = 'Date'
    
    # Reset the index if you want the dates as a column
    df_transposed.reset_index(inplace=True)
    
    df_transposed["TVL"] = df_transposed["Total"]
    
    # Only keep the columns we need
    df_transposed = df_transposed[["Date", "TVL"]]
    
    df_transposed["Date"] = pd.to_datetime(df_transposed["Date"], format="%d/%m/%Y")
    
    # Load prices
    df_prices = pd.read_csv("data/" + coin + "-USD.csv", parse_dates=['Date'])
    
    # Merge together with prices
    df_merged = pd.merge(df_transposed, df_prices, on='Date', how='inner')
    
    print(df_merged.head())
    
    df_merged.to_csv("data/" + coin + ".csv")

        Date          TVL         Open         High          Low        Close  \
0 2021-03-21  64933228.56  58309.91406  58767.89844  56005.61719  57523.42188   
1 2021-03-22  65842440.83  57517.89063  58471.48047  54288.15625  54529.14453   
2 2021-03-23   62926579.2  54511.66016  55985.44141  53470.69531  54738.94531   
3 2021-03-24  62607414.24  54710.48828  57262.38281  52514.33203  52774.26563   
4 2021-03-26   60169756.0  51683.01172  55137.31250  51579.85547  55137.31250   

     Adj Close        Volume  
0  57523.42188  5.194341e+10  
1  54529.14453  5.652145e+10  
2  54738.94531  5.643502e+10  
3  52774.26563  7.056722e+10  
4  55137.31250  5.665220e+10  
        Date               TVL        Open        High         Low  \
0 2019-05-10  377496114.965952  170.312973  175.885269  168.991425   
1 2019-05-11  374509695.161407  173.139313  203.466644  173.139313   
2 2019-05-12  403051737.051272  194.163025  201.881485  183.653931   
3 2019-05-13  407686685.188457  187.417923  205

### Add influence and sentiments

In [7]:
coins = ["BTC", "ETH", "SOL", "SUI"]

for coin in coins:
    # Read the main coin data into a DataFrame
    df = pd.read_csv("data/" + coin + ".csv", parse_dates=["Date"])

    # Load sentiment and influence
    df_sentiment = pd.read_csv("data/sentiment_influence.csv", parse_dates=['_24h_interval'])
    
    # Rename column
    df_sentiment.rename(columns={"_24h_interval" : "Date"}, inplace=True)

    # Merge together with prices
    df_merged = pd.merge(df, df_sentiment, on='Date', how='inner')

    df_merged.to_csv("data/" + coin + ".csv")


### Do the same for 12h intervals

#### ETH and SOL in 12 hour intervals

In [17]:
coins = ["ETH", "SOL"]

for coin in coins:
    # Read the CSV data into a DataFrame
    df = pd.read_csv("data/" + coin + ".csv")
    df = df[["Date", "TVL"]]
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index("Date")
    
    df = df.resample('12H').interpolate()
    
    print(df.head(5))

    df.to_csv("data/" + coin + "_TVL_12h.csv")

                              TVL
Date                             
2020-01-01 00:00:00  6.031933e+08
2020-01-01 12:00:00  6.029450e+08
2020-01-02 00:00:00  6.026967e+08
2020-01-02 12:00:00  5.999431e+08
2020-01-03 00:00:00  5.971895e+08
                              TVL
Date                             
2021-03-18 00:00:00  1.489888e+08
2021-03-18 12:00:00  1.510965e+08
2021-03-19 00:00:00  1.532043e+08
2021-03-19 12:00:00  1.504476e+08
2021-03-20 00:00:00  1.476909e+08


In [20]:
coins = ["ETH", "SOL"]

for coin in coins:
    # Read the CSV data into a DataFrame
    df = pd.read_csv("data/" + coin + "_TVL_12h.csv", parse_dates=['Date'])

    # Load prices
    df_prices = pd.read_csv("data/" + coin + "-USD_12h.csv", parse_dates=['Date'])
    
    # Load sentiments
    df_sentiment = pd.read_csv("data/sentiment_influence_12h.csv", parse_dates=['Date'])

    # Merge together with prices
    df_merged = pd.merge(df, df_prices, on='Date', how='inner')
    
    # Merge together with sentiments
    df_merged = pd.merge(df_merged, df_sentiment, on='Date', how='inner')

    print(df_merged.head())

    df_merged.to_csv("data/" + coin + "_12h.csv")

                 Date           TVL    open    high     low  Adj. Close  \
0 2020-01-01 00:00:00  6.031933e+08  129.16  129.19  128.68      128.87   
1 2020-01-01 12:00:00  6.029450e+08  130.74  131.61  130.70      131.61   
2 2020-01-02 00:00:00  6.026967e+08  130.72  130.78  130.27      130.67   
3 2020-01-02 12:00:00  5.999431e+08  129.52  129.99  129.52      129.96   
4 2020-01-03 00:00:00  5.971895e+08  127.19  127.60  126.82      127.07   

       Volume    close_time      quote_av  trades  tb_base_av  tb_quote_av  \
0  7769.17336  1.577840e+12  1.000930e+06    2504  4149.93345  534619.3390   
1  4163.30667  1.577880e+12  5.460708e+05    2862  2614.85418  343039.9585   
2  3772.66670  1.577930e+12  4.925267e+05    2565  2094.53022  273427.2828   
3  3718.46074  1.577970e+12  4.825412e+05    2625  1917.82172  248879.0704   
4  7303.34389  1.578010e+12  9.289197e+05    3476  3857.27760  490617.7394   

   ignore  sentiment_score  normalized_influence_robust  \
0       0         0.4

### SUI into 6 hour intervals

In [18]:
coins = ["SUI"]

for coin in coins:
    # Read the CSV data into a DataFrame
    df = pd.read_csv("data/" + coin + ".csv")
    df = df[["Date", "TVL"]]
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index("Date")

    df = df.resample('6H').interpolate()

    print(df.head(5))

    df.to_csv("data/" + coin + "_TVL_6h.csv")

                              TVL
Date                             
2023-05-06 00:00:00  3.275880e+07
2023-05-06 06:00:00  3.357236e+07
2023-05-06 12:00:00  3.438593e+07
2023-05-06 18:00:00  3.519949e+07
2023-05-07 00:00:00  3.601305e+07


In [22]:
coins = ["SUI"]

for coin in coins:
    # Read the CSV data into a DataFrame
    df = pd.read_csv("data/" + coin + "_TVL_6h.csv", parse_dates=['Date'])

    # Load prices
    df_prices = pd.read_csv("data/" + coin + "-USD_6h.csv", parse_dates=['Date'])

    # Load sentiments
    df_sentiment = pd.read_csv("data/sentiment_influence_6h.csv", parse_dates=['Date'])

    # Merge together with prices
    df_merged = pd.merge(df, df_prices, on='Date', how='inner')

    # Merge together with sentiments
    df_merged = pd.merge(df_merged, df_sentiment, on='Date', how='inner')

    print(df_merged.head())

    df_merged.to_csv("data/" + coin + "_6h.csv")

                 Date           TVL    open    high     low  Adj. Close  \
0 2023-05-06 00:00:00  3.275880e+07  1.3067  1.3538  1.2902      1.3430   
1 2023-05-06 06:00:00  3.357236e+07  1.3304  1.3607  1.3250      1.3550   
2 2023-05-06 12:00:00  3.438593e+07  1.3243  1.3377  1.3062      1.3112   
3 2023-05-06 18:00:00  3.519949e+07  1.3119  1.3248  1.3018      1.3217   
4 2023-05-07 00:00:00  3.601305e+07  1.3299  1.3469  1.3162      1.3312   

      Volume    close_time     quote_av  trades  tb_base_av  tb_quote_av  \
0  3705766.0  1.683330e+12  4897055.164   12470   2139806.1  2837623.485   
1  3220886.3  1.683360e+12  4336976.950   11491   1975391.3  2660873.397   
2  3169702.5  1.683380e+12  4189969.082   12296   1487455.4  1969046.954   
3  1720384.3  1.683400e+12  2258266.296    8706    888702.5  1167548.102   
4  3435000.4  1.683420e+12  4570773.156   10706   1448952.0  1932118.872   

   ignore  sentiment_score  normalized_influence_robust  \
0       0         0.413783       

#### BTC into 12 hour intervals without TVL

In [23]:
coins = ["BTC"]

for coin in coins:
    # Load prices
    df_prices = pd.read_csv("data/" + coin + "-USD_12h.csv", parse_dates=['Date'])

    # Load sentiments
    df_sentiment = pd.read_csv("data/sentiment_influence_12h.csv", parse_dates=['Date'])

    # Merge together with sentiments
    df_merged = pd.merge(df_prices, df_sentiment, on='Date', how='inner')

    print(df_merged.head())

    df_merged.to_csv("data/" + coin + "_12h.csv")

                 Date     open     high      low  Adj. Close       Volume  \
0 2020-01-01 00:00:00  7195.24  7196.25  7175.46     7177.02   511.814901   
1 2020-01-01 12:00:00  7197.20  7233.33  7196.15     7225.59   868.753580   
2 2020-01-02 00:00:00  7200.77  7212.50  7185.46     7211.02   391.352377   
3 2020-01-02 12:00:00  7139.73  7163.40  7139.03     7158.29   794.030497   
4 2020-01-03 00:00:00  6965.49  6966.47  6912.88     6937.00  1106.732043   

     close_time     quote_av  trades  tb_base_av  tb_quote_av  ignore  \
0  1.577840e+12  3675856.579    7640  226.153126  1624289.190       0   
1  1.577880e+12  6269445.539    9402  504.454945  3640707.643       0   
2  1.577930e+12  2817911.989    5470  219.086231  1577568.039       0   
3  1.577970e+12  5679917.260    8754  419.734423  3001938.644       0   
4  1.578010e+12  7682684.252   11910  540.409116  3752333.710       0   

   sentiment_score  normalized_influence_robust  normalized_influence_minmax  \
0         0.491250